In [ ]:
import cv2
import numpy
import numpy as np
import serial
import math
import matplotlib.pyplot as plt
import time

In [2]:
# 打开摄像机，不轻易运行
capture=cv2.VideoCapture(1)
# ser.close()
# ser = serial.Serial('COM10', 115200)

In [3]:
# 定义字符串的转换方式
# 先定义最大速度
max_speed=10
def convert_to_two_digits(number):
    if 10 > number >= 0:
        return '+0' + str(number)
    if 100 > number >= 10:
        return '+' + str(min(number,max_speed))
    if 0 > number > -10:
        return '-0' + str(number)[1]
    if -10 >= number > -100:
        return '-'+str(min(abs(number),max_speed))
# 在整个图片中获取红色点
# 在整个image所获取的图片中[:,:,1]是绿色图片,[:,:,2]是红色图片
def get_red_point(capture):
    _,image=capture.read()
    # 找到红色的点
    # image_red_true=image[:,:,2]
    # image_back=(image[:,:,1]+image[:,:,0])/2
    # image_red=image_red_true-image_back
    # image_red=image[:,:,2]
    image_red_true=image[:,:,1].astype(np.float32)
    image_back=(image[:,:,2].astype(np.float32)+image[:,:,0].astype(np.float32))/2
    image_back=image_back.astype(np.float32)
    image_red=np.abs(image_red_true-image_back)
    # image_red=image[:,:,1]
    image_red=image_red.astype(np.uint8)
    red = np.array(np.argwhere(image_red>70))
    # red = np.array(np.argwhere(image_red>240))
    # 红色中心点坐标
    if red.shape[0]>2:
        red_center=np.ones(2)
        red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
        red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
    else:
        pass
# # 将红点移动到目标点
# def move_to_the_point(dis_aim,speed,aim_center,sleep_time):
#     dis_center=12
#     speed=speed/10
#     # while True:
#     while dis_center>dis_aim:
#         get_red_point(capture)
#         dis_center=np.linalg.norm(aim_center-red_center)
#         rotate=(aim_center-red_center)*speed
#         rotate=-rotate
#         # 第1个数正上负下  第2个数，正左负右
#         data = convert_to_two_digits(int(rotate[0]))+convert_to_two_digits(int(rotate[1]))
#         encoded_data = data.encode()
#         ser.write(encoded_data)
#         time.sleep(sleep_time)
# # 蜂鸣器响3秒
# def buzzer_open():
#     data = "ffffff"
#     encoded_data = data.encode()
#     ser.write(encoded_data)

In [4]:
# 定义卷积核
kernel = np.array([
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],
[0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0],
[0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0],
[0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0],
])*0.01
a=11
b=11

In [5]:
# 红 绿 蓝
# '''
_,image1=capture.read()
# 对于所获得的图像，首先计算他的几何中心
# 先转为灰度图
gray1=cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
# 获取边界
edges=cv2.Canny(gray1, 240, 200)
# 打开开关
red_center=np.ones(2)
red_center[0]=240
red_center[1]=320
while True:# 首先我们读取摄像头的图片
    _,image=capture.read()
    # 对于所获得的图像，首先计算他的几何中心
    # 先转为灰度图
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges_show=gray
    # 获取边界
    image_red_true=image[:,:,1].astype(np.float32)
    image_back=(image[:,:,2].astype(np.float32)+image[:,:,0].astype(np.float32))/2
    image_back=image_back.astype(np.float32)
    image_red=np.abs(image_red_true-image_back)
    # image_red=image[:,:,1]
    image_red=image_red.astype(numpy.uint8)
    red = np.array(np.argwhere(image_red>70))
    circle= np.array(np.where(edges))
    # 获取红色中心点的坐标
    # if red.shape[0]>2:
    #     red_center=np.ones(2)
    #     red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
    #     red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
    # else:
    #     red_center=np.ones(2)
    #     red_center[0]=240
    #     red_center[1]=320
    get_red_point(capture)
    # 获得边框中心点的坐标
    circle_center=np.ones(2)
    circle_center[0]=int(np.sum(circle[:,0])/(np.size(circle[:,0])+0.001))
    circle_center[1]=int(np.sum(circle[:,1])/(np.size(circle[:,1])+0.001))
    # 之后，我们利用卷积操作，计算我们的中心点的边缘与边框
    # 首先，创建一个同等大小的矩阵
    red_matrix=np.zeros((480,640))
    # 叠加点的图像
    if (int(red_center[0]-a)>0)&(int(red_center[0]+b)<480)&(int(red_center[1]-a)>0)&(int(red_center[1]+b)<640):
        red_matrix[int(red_center[0]-a):int(red_center[0]+b),int(red_center[1]-a):int(red_center[1]+b)]=kernel
        edges_show[int(red_center[0]-a):int(red_center[0]+b),int(red_center[1]-a):int(red_center[1]+b)]=kernel*25400
    else:
        red_matrix[240-a:240+b,320-a:320+b]=kernel
        edges_show[240-a:240+b,320-a:320+b]=kernel*25400
    # 定义得分
    score=np.sum(edges*red_matrix)
    # 打印得分
    # '''
    gray[0,0]=score
    cv2.imshow('image_red', image_red)
    cv2.imshow('image', image)
    cv2.imshow('image_red_true', image[:,:,1])
    cv2.imshow('edges', edges)
    cv2.imshow('score', gray)
    cv2.imshow('edges_show', edges_show)
    cv2.imshow('red_matrix', red_matrix*20000)
    cv2.waitKey(1)
    # '''
#'''

KeyboardInterrupt: 

In [20]:
np.save('Photo_Data\All_photo\\red_matrix.npy', red_matrix)